In [ ]:
# รายการประเภท inquiry แต่ละภาษา
categories = {
    'English' : [
        "General Inquiry",
        "Estimated Cost",
        "Contact My Doctor at Bangkok Hospital Pattaya",
        "Other"
    ],

    'Thai' : [
        "สอบถามทั่วไป",
        "ค่าใช้จ่าย",
        "ติดต่อกับหมอประจำตัวที่โรงพยาบาลกรุงเทพพัทยา",
        "อื่นๆ"
    ],

    'Russia' : [
        "Общий запрос",
        "Узнать про цену",
        "Написать врачу",
        "Другое"
    ],
    'Arabic' : [
        "General Inquiry",
        "Estimated Cost",
        "Contact My Doctor at Bangkok Hospital Pattaya",
        "Other"
    ],

    'Chinese' : [
        "普通咨询",
        "预估价格咨询",
        "联系芭提雅曼谷医院医生",
        "其他"
    ],

    'German' : [
        "Allgemeine Anfrage",
        "Vorraussichtliche Kosten",
        "Arzt im Bangkok Hospital Pattaya kontaktieren",
        "Andere"
    ]
}

In [ ]:
# หา inquiry
import pandas as pd
import glob
import os
from collections import defaultdict

# สร้าง dictionary สำหรับสรุปผล
summary = defaultdict(lambda: defaultdict(int))

# กำหนดโฟลเดอร์ที่เก็บไฟล์ CSV
folder_path = "./csv_folder"  # เปลี่ยนตามตำแหน่งจริง
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

for file in csv_files:
    # อ่านไฟล์
    df = pd.read_csv(file)
    df.columns = df.columns.str.replace('\ufeff', '').str.strip('"')

    col_name = df.columns[0]  # สมมุติว่าใช้คอลัมน์แรกเสมอ

    # ตรวจจับภาษา (จากชื่อไฟล์)
    if "-en" in file:
        lang = "English"
    elif "-th" in file:
        lang = "Thai"
    elif "-ru" in file:
        lang = "Russia"
    elif "-de" in file:
        lang = "German"
    elif "-ar" in file:
        lang = "Arabic"
    elif "-zh" in file:
        lang = "Chinese"
    else:
        continue  # ข้ามไฟล์ที่ไม่มีภาษารองรับ

    # ตรวจสอบว่าเรามีหมวดหมู่ของภาษานี้หรือไม่
    # if lang not in categories:
    #     continue

    for cat in categories[lang]:
        count = df[col_name].astype(str).str.strip().eq(cat).sum()
        summary[lang][cat] += count

# แสดงผลลัพธ์
for lang in summary:
    print(f"\nภาษา: {lang}")
    if lang == "Russia" or lang == "thai" or lang == "German":
        for cat, count in summary[lang].items():
            if cat == "Общий запрос" or cat == "Allgemeine Anfrage":
                print(f" - {cat} (General Inquiry): {count}")
            elif cat == "Узнать про цену" or cat == "Vorraussichtliche Kosten":
                print(f" - {cat} (Estimated Cost): {count}")
            elif cat == "Написать врачу":
                print(f" - {cat} (Contact My Doctor at BPH): {count}")
            elif cat == "Другое" or cat == "Andere":
                print(f" - {cat} (Other): {count}")
            else:
                print(f" - {cat}: {count}")
    else :
        for cat, count in summary[lang].items():
            print(f" - {cat}: {count}")

In [ ]:
# หา appointment & appointment recommended
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os

folder_path = "./csv_folder"
langs = ["ar", "de", "en", "ru", "th", "zh"]
lang_summary = {lang: {"appointment count": 0, "appointment recommended count": 0} for lang in langs}
total_all_col_rec, total_all_col = 0, 0

recommended_files = glob.glob(os.path.join(folder_path, "*appointment-recommended*.csv"))
normal_files = [f for f in glob.glob(os.path.join(folder_path, "*appointment*.csv"))
                if "appointment-recommended" not in os.path.basename(f)]

# อ่านไฟล์ recommended
for file in recommended_files:
    df = pd.read_csv(file)
    df.columns = df.columns.str.strip().str.replace('\ufeff', '')
    col_name = df.columns[1]
    count = len(df[col_name])
    total_all_col_rec += count

    for lang in langs:
        if f"-{lang}" in file:
            lang_summary[lang]["appointment recommended count"] += count
            break

# อ่านไฟล์ปกติ
for file in normal_files:
    df = pd.read_csv(file)
    df.columns = df.columns.str.strip().str.replace('\ufeff', '')
    col_name = df.columns[1]
    count = len(df[col_name])
    total_all_col += count

    for lang in langs:
        if f"-{lang}" in file:
            lang_summary[lang]["appointment count"] += count
            break

# เติมข้อมูลใน DataFrame สำหรับ plot
data_frame_for_plot = {
    "lang": [],
    "appointment count": [],
    "appointment recommended count": []
}

for lang in langs:
    data_frame_for_plot["lang"].append(lang)
    data_frame_for_plot["appointment count"].append(lang_summary[lang]["appointment count"])
    data_frame_for_plot["appointment recommended count"].append(lang_summary[lang]["appointment recommended count"])

# เพิ่ม total
data_frame_for_plot["lang"].append("total")
data_frame_for_plot["appointment count"].append(total_all_col)
data_frame_for_plot["appointment recommended count"].append(total_all_col_rec)

# ✅ สร้าง DataFrame ก่อนนำไป plot
df_plot = pd.DataFrame(data_frame_for_plot)
# แยก total ออกก่อน (ถ้ามี)
df_total = df_plot[df_plot['lang'] == 'total']
df_plot = df_plot[df_plot['lang'] != 'total']

# สร้างคอลัมน์รวม
df_plot['total'] = df_plot['appointment count'] + df_plot['appointment recommended count']

# เรียงลำดับตามค่ารวม
df_plot = df_plot.sort_values(by='total', ascending=False).reset_index(drop=True)

# ลบคอลัมน์ total ที่ใช้ช่วยเรียง
df_plot = df_plot.drop(columns=['total'])

# ต่อแถว total กลับ
df_plot = pd.concat([df_plot, df_total], ignore_index=True)


# --- Plot ---
x = df_plot['lang']
width = 0.35
x_pos = range(len(x))

plt.figure(figsize=(10,6))
bars1 = plt.bar(x_pos, df_plot['appointment count'], width=width, label='Appointment', color='skyblue')
bars2 = plt.bar([p + width for p in x_pos], df_plot['appointment recommended count'], width=width, label='Recommended', color='orange')

# เพิ่มตัวเลขบนแท่งกราฟ
for bar in bars1:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.5, str(int(height)), ha='center', va='bottom')

for bar in bars2:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.5, str(int(height)), ha='center', va='bottom')

plt.xticks([p + width / 2 for p in x_pos], x)
plt.xlabel("Language")
plt.ylabel("Count")
plt.title("Appointment vs Recommended by Language")
plt.legend()
plt.tight_layout()

print(df_plot)
plt.show()
df_plot.to_csv("summary_report.csv", index=False)


In [ ]:
#นับ feedback packages
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os

folder_path = "./csv_folder"
total_all_col_feed, total_all_col_pack = 0, 0 
feedback = glob.glob(os.path.join(folder_path, "*feedback*.csv"))
packages = glob.glob(os.path.join(folder_path, "*packages*.csv"))


# อ่านไฟล์ recommended
for file in feedback:
    df = pd.read_csv(file)
    df.columns = df.columns.str.strip().str.replace('\ufeff', '')
    col_name = df.columns[0]
    count = len(df[col_name])
    total_all_col_feed += count

# อ่านไฟล์ปกติ
for file in packages:
    df = pd.read_csv(file)
    df.columns = df.columns.str.strip().str.replace('\ufeff', '')
    col_name = df.columns[0]
    count = len(df[col_name])
    total_all_col_pack += count

print('feedback:', total_all_col_feed)
print('packages:', total_all_col_pack)


In [ ]:
# plot graph Type of Email by month
import matplotlib.pyplot as plt
from collections import defaultdict

# translations สำหรับแปลงชื่อ category เป็นภาษาอังกฤษ
translations = {
    # Russian
    "Общий запрос": "General Inquiry",
    "Узнать про цену": "Estimated Cost",
    "Написать врачу": "Contact My Doctor at Bangkok Hospital Pattaya",
    "Другое": "Other",
    # German
    "Allgemeine Anfrage": "General Inquiry",
    "Vorraussichtliche Kosten": "Estimated Cost",
    "Arzt im Bangkok Hospital Pattaya kontaktieren": "Contact My Doctor at Bangkok Hospital Pattaya",
    "Andere": "Other",
    # Thai
    "สอบถามทั่วไป": "General Inquiry",
    "ค่าใช้จ่าย": "Estimated Cost",
    "ติดต่อกับหมอประจำตัวที่โรงพยาบาลกรุงเทพพัทยา": "Contact My Doctor at Bangkok Hospital Pattaya",
    "อื่นๆ": "Other",
    # Arabic
    "General Inquiry": "General Inquiry",
    "Estimated Cost": "Estimated Cost",
    "Contact My Doctor at Bangkok Hospital Pattaya": "Contact My Doctor at Bangkok Hospital Pattaya",
    "Other": "Other",
    # Chinese
    "普通咨询": "General Inquiry",
    "预估价格咨询": "Estimated Cost",
    "联系芭提雅曼谷医院医生": "Contact My Doctor at Bangkok Hospital Pattaya",
    "其他": "Other",
    # English already in English
    "General Inquiry": "General Inquiry",
    "Estimated Cost": "Estimated Cost",
    "Contact My Doctor at Bangkok Hospital Pattaya": "Contact My Doctor at Bangkok Hospital Pattaya",
    "Other": "Other",
}

# รวม count โดยใช้ชื่อหมวดหมู่ภาษาอังกฤษ
global_summary = defaultdict(int)

for lang in summary:
    for cat, count in summary[lang].items():
        eng_cat = translations.get(cat, cat)  # แปลงเป็นอังกฤษ ถ้าไม่มีให้ใช้ชื่อเดิม
        global_summary[eng_cat] += count
    
global_summary.pop("", None)  # ถ้าไม่มี key จะไม่เกิด error

# เรียงมากไปน้อย
sorted_items = sorted(global_summary.items(), key=lambda x: x[1], reverse=True)
categories_list = [item[0] for item in sorted_items]
categories_list.append('Package Inquiry')
categories_list.append('Feedback & Suggestion')
categories_list.append('Appointment')
categories_list.append('Appointment Recommended')
categories_list.append('Web Commerce')
counts = [item[1] for item in sorted_items]
counts.append(total_all_col_pack)
counts.append(total_all_col_feed)
counts.append(total_all_col)
counts.append(total_all_col_rec)
counts.append(13)

# วาดกราฟ
plt.figure(figsize=(10, 5))
bars = plt.bar(categories_list, counts, color='mediumseagreen')

# ใส่ตัวเลขบนกราฟ
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height + 1, str(height),
             ha='center', va='bottom', fontsize=10)

plt.ylim(0, max(counts) * 1.2)
plt.title("Type of Email by month", fontsize=14)
plt.xlabel("Category (English)", fontsize=12)
plt.ylabel("Total Count", fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()
